In [1]:
# Dependancies
import pandas as pd
import datetime

# Import data sources   
traffic_data = "../Resources/Historic_Incidents_2016_2017.csv"
rain_data = "../Resources/IDCJAC0009_009225_2017_Rainfall.csv"
temp_data = "../Resources/IDCJAC0010_009225_2017_Temp.csv"

In [2]:
# Read in traffic data
data_df=pd.read_csv(traffic_data)

In [3]:
# Filter to Metro only
data_df = data_df.loc[data_df["Operations_Centre"]=="Metro"]

In [4]:
# Remove other columns
new_data = data_df[["WST_Start","Incident_Type","TrafficCondition","Lat","Long"]]


In [5]:
# Filter to selected timerange (01/12 - 31/12)
date_df = new_data.loc[(new_data['WST_Start'] > "2017/01/01") & (new_data['WST_Start'] <= "2017/12/31")]

# Split Time of Incident into Date + Time
date_df[['Date','Time']] = date_df["WST_Start"].str.split(" ",expand=True)

# Convert Date to datetime object for merge
date_df["Date"] = pd.to_datetime(date_df["Date"])


In [6]:
# Reorder Columns for database
date_df = date_df[["Date","Time","Incident_Type","TrafficCondition","Lat","Long"]]

# Rename Columns for datebase
traffic_df = date_df.rename(columns={"Date":"date","Time":"time","Incident_Type":"type","TrafficCondition":"conditions","Lat":"lat","Long":"long"})

In [7]:
# Import and clean rainfall Data
rain_df = pd.read_csv(rain_data)

# remove columns
weather_df = rain_df[["Year", "Month", "Day", "Rainfall amount (millimetres)"]]

In [8]:
# import Temperature Data
temp_df = pd.read_csv(temp_data)

In [9]:
# Merge Rain and Temp into one df
weather_df["Maximum temperature (Degree C)"] = temp_df["Maximum temperature (Degree C)"]

In [10]:
# Combine Time columns into one Date object
weather_df["Date"] = pd.to_datetime(weather_df[['Year','Month','Day']])

In [11]:
# Reorder columns for database
weather_df = weather_df[["Date","Rainfall amount (millimetres)","Maximum temperature (Degree C)"]]

# Rename columns for database
weather_df = weather_df.rename(columns={"Date":"date","Rainfall amount (millimetres)":"rainfall","Maximum temperature (Degree C)":"temperature"})

In [12]:
weather_df

,date,rainfall,temperature
0,2017-01-01,0.0,31.3
1,2017-01-02,0.0,37.1
2,2017-01-03,0.0,40.0
3,2017-01-04,0.0,40.9
4,2017-01-05,0.0,24.9
...,...,...,...
360,2017-12-27,0.0,25.2
361,2017-12-28,0.0,25.8
362,2017-12-29,0.0,28.3
363,2017-12-30,0.0,34.7


In [13]:
traffic_df

,date,time,type,conditions,lat,long
0,2017-01-01,16:59:00,Road Crash,All Lanes Open,-32.065714,115.894297
2,2017-01-02,04:50:59,Road Crash,All Lanes Open,-31.674911,115.711201
3,2017-01-02,07:30:00,Road Crash,All Lanes Open,-31.936552,115.838520
4,2017-01-02,08:16:59,Road Crash,All Lanes Open,-31.892157,116.015323
5,2017-01-02,12:13:00,Debris / Trees / Lost Loads,Right Centre Lane(s) Blocked,-31.936639,115.838076
...,...,...,...,...,...,...
7992,2017-12-30,20:03:59,Road Crash,Right Lane(s) Blocked,-31.949386,115.871415
7993,2017-12-30,23:23:00,Break Down / Tow Away,Right Emergency Lane Blocked,-31.927353,115.831052
11391,2017-01-06,14:35:00,Break Down / Tow Away,NaN,-31.949682,115.849312
11392,2017-01-06,15:24:59,Duplicate Entry,NaN,-31.940631,116.015960
